### Methods & Results:

In [1]:
### Loading Libraries into R
### Run this cell before continuing.|
library(repr)
library(tidyverse)
library(tidymodels)
options(repr.matrix.max.rows = 6)

── Attaching packages ─────────────────────────────────────── tidyverse 1.3.0 ──

✔ ggplot2 3.3.2     ✔ purrr   0.3.4
✔ tibble  3.0.3     ✔ dplyr   1.0.2
✔ tidyr   1.1.2     ✔ stringr 1.4.0
✔ readr   1.3.1     ✔ forcats 0.5.0

Warning message:
“package ‘ggplot2’ was built under R version 4.0.1”
Warning message:
“package ‘tibble’ was built under R version 4.0.2”
Warning message:
“package ‘tidyr’ was built under R version 4.0.2”
Warning message:
“package ‘dplyr’ was built under R version 4.0.2”
── Conflicts ────────────────────────────────────────── tidyverse_conflicts() ──
✖ dplyr::filter() masks stats::filter()
✖ dplyr::lag()    masks stats::lag()

Warning message:
“package ‘tidymodels’ was built under R version 4.0.2”
── Attaching packages ────────────────────────────────────── tidymodels 0.1.1 ──

✔ broom     0.7.0      ✔ recipes   0.1.13
✔ dials     0.0.9      ✔ rsample   0.0.7 
✔ infer     0.5.4      ✔ tune      0.1.1 
✔ modeldata 0.0.2      ✔ workflows 0.2.0 
✔ parsnip   0.1.3    

In [5]:
### data is read from online
url <-  "https://raw.githubusercontent.com/Lollipop-27/DCSI100-Project/main/data/music_genre.csv"
data <- read_csv(url)
data
### cut data into smaller amount
cut_data <- stratified_sample

Parsed with column specification:
cols(
  instance_id = col_double(),
  artist_name = col_character(),
  track_name = col_character(),
  popularity = col_double(),
  acousticness = col_double(),
  danceability = col_double(),
  duration_ms = col_double(),
  energy = col_double(),
  instrumentalness = col_double(),
  key = col_character(),
  liveness = col_double(),
  loudness = col_double(),
  mode = col_character(),
  speechiness = col_double(),
  tempo = col_character(),
  obtained_date = col_character(),
  valence = col_double(),
  music_genre = col_character()
)



instance_id,artist_name,track_name,popularity,acousticness,danceability,duration_ms,energy,instrumentalness,key,liveness,loudness,mode,speechiness,tempo,obtained_date,valence,music_genre
<dbl>,<chr>,<chr>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<chr>,<dbl>,<dbl>,<chr>,<dbl>,<chr>,<chr>,<dbl>,<chr>
32894,Röyksopp,Röyksopp's Night Out,27,0.00468,0.652,-1,0.941,0.7920,A#,0.115,-5.201,Minor,0.0748,100.889,04-Apr,0.759,Electronic
46652,Thievery Corporation,The Shining Path,31,0.01270,0.622,218293,0.890,0.9500,D,0.124,-7.043,Minor,0.0300,115.002,04-Apr,0.531,Electronic
30097,Dillon Francis,Hurricane,28,0.00306,0.620,215613,0.755,0.0118,G#,0.534,-4.617,Major,0.0345,127.994,04-Apr,0.333,Electronic
⋮,⋮,⋮,⋮,⋮,⋮,⋮,⋮,⋮,⋮,⋮,⋮,⋮,⋮,⋮,⋮,⋮,⋮
39767,Berner,Lovin' Me (feat. Smiggz),51,0.00597,0.693,189483,0.763,0,D,0.143,-5.443,Major,0.1460,131.079,04-Apr,0.395,Hip-Hop
57944,The-Dream,Shawty Is Da Shit,65,0.08310,0.782,262773,0.472,0,G,0.106,-5.016,Minor,0.0441,75.886,04-Apr,0.354,Hip-Hop
63470,Naughty By Nature,Hip Hop Hooray,67,0.10200,0.862,267267,0.642,0,F#,0.272,-13.652,Minor,0.1010,99.201,04-Apr,0.765,Hip-Hop


ERROR: Error in eval(expr, envir, enclos): object 'stratified_sample' not found


At first, the data set we selected had about 50000 data, and we all thought that these data were too many to use. Therefore, we choose to cut each music genre by 25% in order to reduce the amount of data. After discussing with the teaching assistant, we choose to use stratified_sample() to compress our data into a smaller amount yet still keeping the same 10 music genres. 

In [9]:
### removing the columns we don't need
data <- mutate(data,mode = as.factor(mode),
                   music_genre = as.factor(music_genre))

data_select <- data %>%
            select(-instance_id, -key,-obtained_date)


Before we removed any columns that we don't need we used mutate() to make mode, music_genre into factor and tempo as a double. With factors and doubles, we can use these data in our models.

We all agreed that instance_id, key, and obtained_date are columns that we will not be needing in our model. Therefore we used select to remove these 3 columns. 

In [9]:
### spliting the data into training set and testing set, 75% of the data in the training set
data_split <- initial_split(data_select, prop = 0.75, strata = music_genre) 
data_train <- training(data_split)   
data_test <- testing(data_split)

Warning message:
“Too little data to stratify. Unstratified resampling will be used.”


We chose to split at 75: 25 because we all agreed that it would be a split, which will tell us whether our model was over-fitted or under-fitted. We will have the appropriate amount of data for training and testing. 

In [ ]:
### Table for each Genre
Electronic <- data_train %>%
                filter(music_genre == "Electronic") %>%
                group_by(music_genre) %>%
                summarize(Electronic_popularity = mean(popularity, na.ra = True),
                         Electronic_acousticness = mean(acousticness, na.ra = True),
                         Electronic_danceability = mean(danceability, na.ra = True),
                         Electronic_duration_ms = mean(duration_ms, na.ra = True),
                         Electronic_energy = mean(energy, na.ra = True),
                         Electronic_instrumentalness = mean(instrumentalness, na.ra = True),
                         Electronic_liveness = mean(liveness, na.ra = True),
                         Electronic_loudness = mean(loudness, na.ra = True),
                         Electronic_speechiness = mean(speechiness, na.ra = True),
                         Electronic_tempo = mean(tempo, na.ra = True),
                         Electronic_valence = mean(valence, na.ra = True))
Electronic

Anime <- data_train %>%
                filter(music_genre == "Anime") %>%
                group_by(music_genre) %>%
                summarize(Anime_popularity = mean(popularity, na.ra = True),
                         Anime_acousticness = mean(acousticness, na.ra = True),
                         Anime_danceability = mean(danceability, na.ra = True),
                         Anime_duration_ms = mean(duration_ms, na.ra = True),
                         Anime_energy = mean(energy, na.ra = True),
                         Anime_instrumentalness = mean(instrumentalness, na.ra = True),
                         Anime_liveness = mean(liveness, na.ra = True),
                         Anime_loudness = mean(loudness, na.ra = True),
                         Anime_speechiness = mean(speechiness, na.ra = True),
                         Anime_tempo = mean(tempo, na.ra = True),
                         Anime_valence = mean(valence, na.ra = True))
Anime

Jazz <- data_train %>%
                filter(music_genre == "Jazz") %>%
                group_by(music_genre) %>%
                summarize(Jazz_popularity = mean(popularity, na.ra = True),
                         Jazz_acousticness = mean(acousticness, na.ra = True),
                         Jazz_danceability = mean(danceability, na.ra = True),
                         Jazz_duration_ms = mean(duration_ms, na.ra = True),
                         Jazz_energy = mean(energy, na.ra = True),
                         Jazz_instrumentalness = mean(instrumentalness, na.ra = True),
                         Jazz_liveness = mean(liveness, na.ra = True),
                         Jazz_loudness = mean(loudness, na.ra = True),
                         Jazz_speechiness = mean(speechiness, na.ra = True),
                         Jazz_tempo = mean(tempo, na.ra = True),
                         Jazz_valence = mean(valence, na.ra = True))
Jazz

Alternative <- data_train %>%
                filter(music_genre == "Alternative") %>%
                group_by(music_genre) %>%
                summarize(Alternative_popularity = mean(popularity, na.ra = True),
                         Alternative_acousticness = mean(acousticness, na.ra = True),
                         Alternative_danceability = mean(danceability, na.ra = True),
                         Alternative_duration_ms = mean(duration_ms, na.ra = True),
                         Alternative_energy = mean(energy, na.ra = True),
                         Alternative_instrumentalness = mean(instrumentalness, na.ra = True),
                         Alternative_liveness = mean(liveness, na.ra = True),
                         Alternative_loudness = mean(loudness, na.ra = True),
                         Alternative_speechiness = mean(speechiness, na.ra = True),
                         Alternative_tempo = mean(tempo, na.ra = True),
                         Alternative_valence = mean(valence, na.ra = True))
Alternative

Country <- data_train %>%
                filter(music_genre == "Country") %>%
                group_by(music_genre) %>%
                summarize(Country_popularity = mean(popularity, na.ra = True),
                         Country_acousticness = mean(acousticness, na.ra = True),
                         Country_danceability = mean(danceability, na.ra = True),
                         Country_duration_ms = mean(duration_ms, na.ra = True),
                         Country_energy = mean(energy, na.ra = True),
                         Country_instrumentalness = mean(instrumentalness, na.ra = True),
                         Country_liveness = mean(liveness, na.ra = True),
                         Country_loudness = mean(loudness, na.ra = True),
                         Country_speechiness = mean(speechiness, na.ra = True),
                         Country_tempo = mean(tempo, na.ra = True),
                         Country_valence = mean(valence, na.ra = True))
Country

Rap <- data_train %>%
                filter(music_genre == "Rap") %>%
                group_by(music_genre) %>%
                summarize(Rap_popularity = mean(popularity, na.ra = True),
                         Rap_acousticness = mean(acousticness, na.ra = True),
                         Rap_danceability = mean(danceability, na.ra = True),
                         Rap_duration_ms = mean(duration_ms, na.ra = True),
                         Rap_energy = mean(energy, na.ra = True),
                         Rap_instrumentalness = mean(instrumentalness, na.ra = True),
                         Rap_liveness = mean(liveness, na.ra = True),
                         Rap_loudness = mean(loudness, na.ra = True),
                         Rap_speechiness = mean(speechiness, na.ra = True),
                         Rap_tempo = mean(tempo, na.ra = True),
                         Rap_valence = mean(valence, na.ra = True))
Rap

Blues <- data_train %>%
                filter(music_genre == "Blues") %>%
                group_by(music_genre) %>%
                summarize(Blues_popularity = mean(popularity, na.ra = True),
                         Blues_acousticness = mean(acousticness, na.ra = True),
                         Blues_danceability = mean(danceability, na.ra = True),
                         Blues_duration_ms = mean(duration_ms, na.ra = True),
                         Blues_energy = mean(energy, na.ra = True),
                         Blues_instrumentalness = mean(instrumentalness, na.ra = True),
                         Blues_liveness = mean(liveness, na.ra = True),
                         Blues_loudness = mean(loudness, na.ra = True),
                         Blues_speechiness = mean(speechiness, na.ra = True),
                         Blues_tempo = mean(tempo, na.ra = True),
                         Blues_valence = mean(valence, na.ra = True))
Blues

Rock <- data_train %>%
                filter(music_genre == "Rock") %>%
                group_by(music_genre) %>%
                summarize(Rock_popularity = mean(popularity, na.ra = True),
                         Rock_acousticness = mean(acousticness, na.ra = True),
                         Rock_danceability = mean(danceability, na.ra = True),
                         Rock_duration_ms = mean(duration_ms, na.ra = True),
                         Rock_energy = mean(energy, na.ra = True),
                         Rock_instrumentalness = mean(instrumentalness, na.ra = True),
                         Rock_liveness = mean(liveness, na.ra = True),
                         Rock_loudness = mean(loudness, na.ra = True),
                         Rock_speechiness = mean(speechiness, na.ra = True),
                         Rock_tempo = mean(tempo, na.ra = True),
                         Rock_valence = mean(valence, na.ra = True))
Rock

Classical <- data_train %>%
                filter(music_genre == "Classical") %>%
                group_by(music_genre) %>%
                summarize(Classical_popularity = mean(popularity, na.ra = True),
                         Classical_acousticness = mean(acousticness, na.ra = True),
                         Classical_danceability = mean(danceability, na.ra = True),
                         Classical_duration_ms = mean(duration_ms, na.ra = True),
                         Classical_energy = mean(energy, na.ra = True),
                         Classical_instrumentalness = mean(instrumentalness, na.ra = True),
                         Classical_liveness = mean(liveness, na.ra = True),
                         Classical_loudness = mean(loudness, na.ra = True),
                         Classical_speechiness = mean(speechiness, na.ra = True),
                         Classical_tempo = mean(tempo, na.ra = True),
                         Classical_valence = mean(valence, na.ra = True))
Classical

Hip_Hop <- data_train %>%
                filter(music_genre == "Hip-Hop") %>%
                group_by(music_genre) %>%
                summarize(Hip_Hop_popularity = mean(popularity, na.ra = True),
                         Hip_Hop_acousticness = mean(acousticness, na.ra = True),
                         Hip_Hop_danceability = mean(danceability, na.ra = True),
                         Hip_Hop_duration_ms = mean(duration_ms, na.ra = True),
                         Hip_Hop_energy = mean(energy, na.ra = True),
                         Hip_Hop_instrumentalness = mean(instrumentalness, na.ra = True),
                         Hip_Hop_liveness = mean(liveness, na.ra = True),
                         Hip_Hop_loudness = mean(loudness, na.ra = True),
                         Hip_Hop_speechiness = mean(speechiness, na.ra = True),
                         Hip_Hop_tempo = mean(tempo, na.ra = True),
                         Hip_Hop_valence = mean(valence, na.ra = True))
Hip_Hop

In [ ]:
# proportion of music genre by mode(major, minor)
mode_proportion_plot <- data_train %>%
ggplot(aes(x = mode, fill = music_genre))+
geom_bar(position = "fill")+
labs(x = "Mode", y = "Proportion of music genre", fill = "music genre") +
ggtitle("Proportion of music genre by mode(major, minor)")
mode_proportion_plot

We visualize the proportion of musical genres by mode or major and minor. Mainly because we wanted to know whether the proportion of major or minor will be higher in a particular genre of music. We found that no music type has a higher proportion, and all music is well balanced. There is much more classical music in major than in minor, which was kind of shocking.

In [ ]:
### Scale dataset
set.seed(1234) 

#find the best k

options(repr.plot.height = 5, repr.plot.width = 6)

data_recipe <- recipe(music_genre ~ popularity + acousticness + danceability + duration_ms + energy + instrumentalness + liveness + loudness + speechiness + valence , data = data_train)


genre_vfold <- vfold_cv(data_train, v = 5, strata = music_genre)


data_spec <- nearest_neighbor(weight_func = "rectangular", neighbors = tune()) %>%
      set_engine("kknn") %>%
      set_mode("classification")


data_results <- workflow() %>%
      add_recipe(data_recipe) %>%
      add_model(data_spec) %>%
      tune_grid(resamples = genre_vfold, grid = 10) %>%
      collect_metrics()

accuracies <- data_results %>%
      filter(.metric == "accuracy")

# cross_val_plot <- accuracies %>%
# ggplot(aes(x = neighbors, y = mean))+
#       geom_point() +
#       geom_line() +
#       labs(x = "Neighbors", y = "Accuracy Estimate") +
#       scale_x_continuous(breaks = seq(0, 15, by = 1)) +  # adjusting the x-axis
#       scale_y_continuous(limits = c(0.0, 0.6)) # adjusting the y-axis
# cross_val_plot



For the data recipe, we choose to use popularity, acousticness, danceability, duration_ms, energy, instrumentalness, liveness, loudness, speechiness, and valence as our predictors to predict the music genre. We choose to make 5 vfold in order to improve the accuracy of our model. For our model, we choose to make a classification model with a tune() neighbour and in a linear form. For our workflow analysis, we used the data_recipe and data_spec we created earlier, since it was a knn model we used fit_resamples() for training with the grid set to 10. We used collect_metrics() to aggregate the mean and standard error from the validation. We had to filter out the accuracy from the .metric column to find the best value for k. We then created a line plot to show what the best k could be for our model. 

In [ ]:
# Set the seed. 
set.seed(9999) 
# build model
data1_spec <- nearest_neighbor(weight_func = "rectangular", neighbors = 14) %>%
       set_engine("kknn") %>%
       set_mode("classification")
data1_spec

data_fit <- workflow() %>%
       add_recipe(data_recipe) %>%
       add_model(data_spec) %>%
       fit(data = data_train)
data_fit

From our plot above we have chosen to use ___ as our k value. Then, we constructed a new model specification with the best value for k. We also formulated a new workflow based on our new specification. 

In [ ]:
#predictions
data_predictions <- predict(data_fit , data_test) %>%
      bind_cols(data_test)
data_predictions

data_metrics <- data_predictions %>%
       metrics(truth = music_genre, estimate = .pred_class)
data_metrics

data_conf_mat<- data_predictions %>%
       conf_mat(truth = music_genre, estimate = .pred_class) 
data_conf_mat

We used our final model to make predictions on the test data set, and use the bind_cols function to add the prediction column to the original test data. Reported the accuracy of this prediction using the metrics function. We also report the confusion matrix using the conf_mat function. 

In [ ]:
# test 1
knn_fit <- data_spec %>% fit(music_genre ~ popularity + acousticness + danceability + duration_ms + energy + instrumentalness + liveness + loudness + speechiness + valence , data = data_train)
new_obs_1 <- tibble(popularity = 33, acousticness = 0.927, danceability = 0.351, duration_ms = 282067, energy = 0.0964, instrumentalness = 0.899, liveness = 0.509, loudness = -20.044, speechiness = 0.0436, valence = 0.246)
class_prediction_1 <-  predict(knn_fit, new_obs_1)


In [ ]:
# test 2
new_obs_2 <- tibble(popularity = 50, acousticness = 0.0145, danceability = 0.567, duration_ms = 268333, energy = 0.3, instrumentalness = 0, liveness = 0.0687, loudness = -8.421, speechiness = 0.0291, valence = 0.196)
class_prediction_2 <-  predict(knn_fit, new_obs_2)


In [ ]:
# test 3
new_obs_2 <- tibble(popularity = 51, acousticness = 0.0341, danceability = 0.634, duration_ms = 248107, energy = 0.716, instrumentalness = 0, liveness = 0.0877, loudness = -6.502, speechiness = 0.128, valence = 0.619)
class_prediction_3 <-  predict(knn_fit, new_obs_3)


The data from the above 3 tests are pulled from the orginal dataset before it was imported into JupyterLab just to test out our model.